---
title: Gamma rhythms models
project:
  type: website
format:
  html:
    code-fold: false
    code-tools: true
jupyter: python 3
number-sections: false
---

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ING

Define the ING model. It's the [HH model](https://raw.githubusercontent.com/Mark-Kramer/BU-MA665-MA666/master/HH_functions.py), with an update to include the inhibitory synaptic dynamics.

In [ ]:
def alphaM(V):
    return (2.5-0.1*(V+65)) / (np.exp(2.5-0.1*(V+65)) -1)

def betaM(V):
    return 4*np.exp(-(V+65)/18)

def alphaH(V):
    return 0.07*np.exp(-(V+65)/20)

def betaH(V):
    return 1/(np.exp(3.0-0.1*(V+65))+1)

def alphaN(V):
    return (0.1-0.01*(V+65)) / (np.exp(1-0.1*(V+65)) -1)

def betaN(V):
    return 0.125*np.exp(-(V+65)/80)

def ing(I0,gI,tauI,T0):
    dt = 0.01;
    T  = int(np.ceil(T0/dt))  # [ms]
    gNa0 = 120   # [mS/cm^2]
    ENa  = 125;  # [mV]
    gK0  = 36;   # [mS/cm^2]
    EK   = -12;  # [mV]
    gL0  = 0.3;  # [mS/cm^2]
    EL   = 10.6; # [mV]

    t = np.arange(0,T)*dt
    V = np.zeros([T,1])
    m = np.zeros([T,1])
    h = np.zeros([T,1])
    n = np.zeros([T,1])

    # Initialize variables to hold the synapse results
    s = np.zeros(T)

    V[0]=-70.0
    m[0]=0.05
    h[0]=0.54
    n[0]=0.34
    s[0]=0.0

    for i in range(0,T-1):
        V[i+1] = V[i] + dt*(gNa0*m[i]**3*h[i]*(ENa-(V[i]+65)) + gK0*n[i]**4*(EK-(V[i]+65)) + gL0*(EL-(V[i]+65)) + I0 + gI*s[i]*(-80 - V[i]))
        m[i+1] = m[i] + dt*(alphaM(V[i])*(1-m[i]) - betaM(V[i])*m[i]);
        h[i+1] = h[i] + dt*(alphaH(V[i])*(1-h[i]) - betaH(V[i])*h[i]);
        n[i+1] = n[i] + dt*(alphaN(V[i])*(1-n[i]) - betaN(V[i])*n[i]);
        s[i+1] = s[i] + dt * (((1 + np.tanh(V[i] / 10)) / 2) * (1 - s[i]) / 0.5 - s[i] / tauI)  # Update s
    return V,s,t

## Begin with the inhibitory synapse turned off.

In [ ]:
I0   = 30;    #Set the input drive to excite the cell.
gI   = 0;     #Turn off inhibitory synapse, to start.
tauI = 10;    #Set decay time of inhibitory synapse to 10ms.
T0   = 100;   #Simulate for 100 ms.

[V,s,t] = ing(I0,gI,tauI,T0)
plt.plot(t,V); plt.xlabel('Time [ms]'); plt.ylabel('Voltage [mV]');

## Turn on the inhibitory synapse.

In [ ]:
gI = 'SOMETHING'
[V,s,t] = ing(I0,gI,tauI,T0);
plt.subplot(2,1,1)
plt.plot(t,V); plt.xlabel('Time [ms]'); plt.ylabel('Voltage [mV]');
plt.subplot(2,1,2)
plt.plot(t,s,'r'); plt.xlabel('Time [ms]'); plt.ylabel('Inhibitory synaptic gate');

**Q:** How does the spiking frequency change?

**Q:** How do the synapses evolve?

**Q:** This model captures 3 experimental observations we discussed in class.  To generate gamma,
1) The cell needs sufficient excitatory drive,
2) The GABA synapse is critical, and
3) Altering the decay time of the inhibitory synapse changes the gamma frequency.

Show that all three observations are captured by this model.